In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm

In [3]:
sys.path.append('../../datasets/mobile-sensing-human-activity-data-set-main/src')
from utils import *
sys.path.append('../../benchmark/utils')
from evaluation import *
sys.path.append('../')
from ClassificationParScoreProfile import *
from DynamicProgramming import *
from FLOSS import *
from Pelt import *
from Window import *
from BayesianOnlineChangePointDetection import *
from Fluss import *

In [4]:
# from src.utils import load_mosad_dataset
df_mosad = load_mosad_dataset()
df_mosad['ts_length']=df_mosad.time_series.apply(len)
df_mosad['cp_length']=df_mosad.change_points.apply(len)
df_mosad.head()

,dataset,routine,subject,sensor,sample_rate,change_points,activities,time_series,ts_length,cp_length
0,Routine1_Subject1_X-Acc,1,1,X-Acc,50,"[738, 1722, 11356, 14124, 20558, 29858, 32736,...","[descend stairs, climb stairs, vacuum, lie, ir...","[-0.1986, -0.454326, -0.5473, -0.496056, -0.48...",59637,12
1,Routine1_Subject1_X-Gyro,1,1,X-Gyro,50,"[738, 1722, 11356, 14124, 20558, 29858, 32736,...","[descend stairs, climb stairs, vacuum, lie, ir...","[0.0174, 0.0358, 0.0431, 0.0358, 0.0168, 0.008...",59637,12
2,Routine1_Subject1_X-Mag,1,1,X-Mag,50,"[738, 1722, 11356, 14124, 20558, 29858, 32736,...","[descend stairs, climb stairs, vacuum, lie, ir...","[-31.02, -30.84, -30.65, -30.58, -31.52, -30.7...",59637,12
3,Routine1_Subject1_Y-Acc,1,1,Y-Acc,50,"[738, 1722, 11356, 14124, 20558, 29858, 32736,...","[descend stairs, climb stairs, vacuum, lie, ir...","[-0.0161, -0.100877, -0.1434, -0.13972, -0.081...",59637,12
4,Routine1_Subject1_Y-Gyro,1,1,Y-Gyro,50,"[738, 1722, 11356, 14124, 20558, 29858, 32736,...","[descend stairs, climb stairs, vacuum, lie, ir...","[-0.0525, -0.0568, -0.0482, -0.0012, 0.0153, 0...",59637,12


In [4]:
mosad=df_mosad[['dataset','change_points','time_series']]

In [5]:
results={}
def evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,algorithm_name):
    # Initialize lists to store metric values for each time series
    covering_scores = []
    f_measure_scores = []
    nab_scores = []
    i=0
    # Iterate over each time series
    for cps_true,cps_found,ts_len in zip(all_cps_true,all_cps_found,all_ts_len):
        i=i+1
        score_covering = covering({0: cps_true}, cps_found, ts_len)
        covering_scores.append(score_covering)
        
        # Calculate F-measure
        score_F = f_measure({0: cps_true}, cps_found, ts_len) 
        f_measure_scores.append(score_F)
        
        # Format true and predicted change points as Series
        true_cp, predicted_cp = formate_data(cps_true, cps_found,ts_len)
    
        # Evaluate change points using the evaluating_change_point function
        score_NAB = evaluating_change_point(true_cp, predicted_cp, metric='nab')
        nab_scores.append(score_NAB)
        # print(score_NAB)

    # Calculate mean scores across all time series
    mean_covering = sum(covering_scores) / len(covering_scores)
    mean_f_measure = sum(f_measure_scores) / len(f_measure_scores)

    # Calculate mean NAB scores
    nab_scores_filtered = [score for score in nab_scores if not np.isnan(score['Standart']) and not np.isinf(score['Standart'])]
    mean_nab_standard = sum(score['Standart'] for score in nab_scores_filtered) / len(nab_scores_filtered)
    mean_nab_lowFP = sum(score['LowFP'] for score in nab_scores_filtered) / len(nab_scores_filtered)
    mean_nab_lowFN = sum(score['LowFN'] for score in nab_scores_filtered) / len(nab_scores_filtered)


    # Print mean scores
    print(f"Mean Covering: {mean_covering}")
    print(f"Mean F-measure: {mean_f_measure}")
    print(f"Mean NAB (Standard): {mean_nab_standard}")
    print(f"Mean NAB (LowFP): {mean_nab_lowFP}")
    print(f"Mean NAB (LowFN): {mean_nab_lowFN}")

    results[algorithm_name]={"Covering":mean_covering,
                                        "F-measure":mean_f_measure,
                                        "NAB (Standard)":mean_nab_standard,
                                        "NAB (LowFP)":mean_nab_lowFP,
                                        "NAB (LowFN)":mean_nab_lowFN
    }


<h3>BinaryClaSPSegmentation</h3>

In [6]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(ts_name,cps_true,ts) in tqdm(mosad.iterrows()):
  cps_found = BinaryClaSPSegmentation().fit_predict(ts)
  all_cps_true.append(cps_true)
  all_cps_found.append(cps_found)
  all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"BinaryClaSPSegmentation")

0it [00:00, ?it/s]

<h3>DynamicProgramming</h3>

In [ ]:
def filter_cp(cp,maxi):
    new_cp=[]
    for pt in cp:
        if pt<maxi:
            new_cp.append(pt)
    return new_cp

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(ts_name,cps_true,ts) in tqdm(mosad.iterrows()):
  cps_found = DynamicProgramming().fit_predict(np.array(ts),len(cps_true))[:-1]
  all_cps_true.append(cps_true)
  all_cps_found.append(cps_found)
  all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"DynamicProgramming")

0it [00:00, ?it/s]

126it [00:02, 48.80it/s]
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_

Mean Covering: 0.8563848617908071
Mean F-measure: 1.0
Mean NAB (Standard): 25.767592592592592
Mean NAB (LowFP): 20.06259259259259
Mean NAB (LowFN): 28.28685185185184


d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Eva

<h3>FLOSS</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(ts_name,cps_true,ts) in tqdm(mosad.iterrows()):
    # Instantiate FLOSS
    floss_detector = FLOSS(n_timepoints=len(ts), window_size=20, n_prerun=500, threshold=0.7, excl_factor=5, verbose=0)
    # Update FLOSS with each time point
    for timepoint in ts:
        floss_detector.update(timepoint)

    cps_found = floss_detector.change_points
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"FLOSS")


10it [00:38,  3.83s/it]

Mean Covering: 0.6849932661331528
Mean F-measure: 0.8438095238095238
Mean NAB (Standard): 9.55
Mean NAB (LowFP): -0.9
Mean NAB (LowFN): 13.032999999999998


<h3>Pelt</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(ts_name,cps_true,ts) in tqdm(mosad.iterrows()):
  cps_found = Pelt().fit_predict(np.array(ts),pen=45)
  all_cps_true.append(cps_true)
  all_cps_found.append(cps_found)
  all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"Pelt")

NameError: name 'mosad' is not defined

<h3>BOCD</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _, (ts_name, window_size, cps_true, ts) in tqdm(tssb.iterrows()):
    # Fit the BinaryClaSPSegmentation algorithm and predict change points
    cps_found = bocd(ts,len(cps_true))
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"BOCD")
results

NameError: name 'mosad' is not defined

<h3>Fluss</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(ts_name,cps_true,ts) in tqdm(mosad.iterrows()):
        # Parameters
    window_size = 50
    n_cps = len(cps_true)
    # Using fluss function
    cps_found = fluss(ts, window_size, n_cps)
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"fluss")

In [ ]:
# Convert dictionary to DataFrame
results_df = pd.DataFrame(results).T.reset_index()

# Rename the 'index' column to 'name_algo'
results_df = results_df.rename(columns={'index': 'name_algo'})
# save the file
results_df.to_csv('results/MOSAD.csv', index=False)

print("DataFrame saved as 'MOSAD.csv'")
# Display the DataFrame
results_df

DataFrame saved as 'MOSAD.csv'


,name_algo,Covering,F-measure,NAB (Standard),NAB (LowFP),NAB (LowFN)
0,DynamicProgramming,0.856385,1.00000,25.767593,20.062593,28.286852
1,FLOSS,0.684993,0.84381,9.550000,-0.900000,13.033000


In [ ]:
results

{'DynamicProgramming': {'Covering': 0.8563848617908071,
  'F-measure': 1.0,
  'NAB (Standard)': 25.767592592592592,
  'NAB (LowFP)': 20.06259259259259,
  'NAB (LowFN)': 28.28685185185184},
 'FLOSS': {'Covering': 0.6849932661331528,
  'F-measure': 0.8438095238095238,
  'NAB (Standard)': 9.55,
  'NAB (LowFP)': -0.9,
  'NAB (LowFN)': 13.032999999999998}}